### Opdracht 
We gaan proberen om een voorspelling te doen over de uitslag van een (verkleinde) voetbalpool om te zien wie er waarschijnlijk gaat winnen. We gaan deze voorspelling doen aan de hand van de  Random Number Generator en het principe vanMonte Carlo Simulatie.

Maak een Monte Carlo Simulatie die meerdere keren (duizenden) de competitie speelt met gebruik van deze tabel van kansen. Hoe vaker je een competitie speelt, hoe nauwkeuriger je voorspelling wordt. Houd bij hoeveel punten elk team scoort in de competitie (3 punten voor winst, 1 punt voor gelijk, 0 voor verlies), en maak een overzicht (ranking) van de teams.

<img src = "tabelMC.PNG">

In [1]:
import random

Definieer een team die ene naam bevat en een lijst van hun mogelijke kansen om te winnen.

In [2]:
clubs = {
    "ajax":{"feyenoord":[65,17,18], "psv":[54,21,25], "fcutrecht":[74,14,12], "willemII":[78,13,9]},
    "feyenoord": {"ajax": [30,21,49], "psv":[37,24,39], "fcutrecht":[51,22,27], "willemII":[60,21,19]},
    "psv": {"ajax": [39,22,39], "feyenoord":[54,22,24], "fcutrecht":[62,20,18], "willemII": [62,22,16]},
    "fcutrecht": {"ajax":[25,14,61], "feyenoord": [37,23,40], "psv":[29,24,47], "willemII": [53,23,25]},
    "willemII": {"ajax": [17, 18, 65], "feyenoord":[20,26,54], "psv":[23,24,53], "fcutrecht":[37,25,38]}
}

Definieer competities waarbij elk team thuis speelt en speelt met de rest van de teams (uitploegen).

In [3]:
def random_number_generator(n):
    "Generating a random number using linear congruential method "
    seed = 20
    a = 3
    c = 3
    m = 100
    
    x=[seed] # list containing n random generated numbers 
    for i in range(1,n+1):
        if i == 1:
            x_next = (a * seed + c) % m
            x.append(x_next)
        else:
            x_next = (a * x[i-1] + c) % m
            x.append(x_next)
    return x


In [4]:
def match(team_lst, rn):
    
    #vb ayax - feyenoord [65, 17, 18]
    if  rn <= team_lst[2]: # rn< 18 
        return 0, 3
    elif team_lst[2] < rn <= (team_lst[2] + team_lst [1]): # rn zit tussen 18 en  35 (18+17)
        return 1,1
    elif  rn >=(team_lst[1]+ team_lst[2]): # rn zit tussen 35 en 100 (35 + 65)
        return 3, 0
    

In [5]:
#list containing the scores of each team


def competitions(hteam, rng):
    scores_teams = {
    "ajax" : 0,
    "feyenoord" : 0,
    "psv" : 0,
    "fcutrecht" : 0,
    "willemII" : 0
}
    counter = 0
    oponents = clubs[hteam]
    for ateam in oponents:
        rn = random.choice(rng)
        ht_point, at_point = match(oponents.get(ateam), rn)# pak de lijst van kansen voor de away team van de ene home team
#             print(ht_point)
#             print(at_point)
#             print("Home/{}: {}, Away:{}:{}".format(hteam, ht_point, ateam, at_point))
        scores_teams[hteam] += ht_point
        scores_teams[ateam] += at_point
        
        counter +=1
#     print(scores_teams)
#     print("\n")
    return scores_teams



In [6]:
rng  = random_number_generator(100) # TODO: generate different random numbers per competitions.

In [7]:
def sort_dict(dictionary):
    # Bron: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    sorted_dict = {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1], reverse= True)}
    return sortedict


In [8]:
def add_rank(hteam, rank_dict, prob_pos_clubs):
    "add ranking into dictionary prob_pos_clubs"
    pos = 1
    for i in rank_dict.keys():
        prob_pos_clubs[i][pos] += 1
        pos +=1

In [9]:
def championships(prob_pos_clubs):
    "Each teams plays in a competition as hometeam"  

    for i in prob_pos_clubs.keys():
        scores_comp = competitions(i, rng)
        rank_comp   = {k: v for k, v in sorted(scores_comp.items(), key=lambda item: item[1], reverse= True)}
#         print("Ranking competion , htmeam : {}".format(i))
#         print(rank_comp)
        add_rank(i, rank_comp, prob_pos_clubs) # TODO functie aanpassen 


In [10]:
prob_pos_clubs = {
        "ajax":{1:0, 2:0, 3:0, 4:0, 5:0},
        "feyenoord": {1:0, 2:0, 3:0, 4:0, 5:0},
        "psv":{1:0, 2:0, 3:0, 4:0, 5:0},
        "fcutrecht":{1:0, 2:0, 3:0, 4:0, 5:0},
        "willemII":{1:0, 2:0, 3:0, 4:0, 5:0}
    }

def MC(n, prob_pos_clubs):
    counter = 0
    while counter  <= n:
        championships(prob_pos_clubs)
        counter +=1
    return prob_pos_clubs

In [37]:
res = MC(1, prob_pos_clubs)
#convert postions values to probability 
for i in res:
    for j in res[i].values():
        j = (j/20)*100
res

{'ajax': {1: 45, 2: 31, 3: 16, 4: 16, 5: 2},
 'feyenoord': {1: 21, 2: 38, 3: 20, 4: 23, 5: 8},
 'psv': {1: 22, 2: 23, 3: 39, 4: 15, 5: 11},
 'fcutrecht': {1: 15, 2: 12, 3: 18, 4: 39, 5: 26},
 'willemII': {1: 7, 2: 6, 3: 17, 4: 17, 5: 63}}

 Convert dictionaries of probability into a dataframe 

In [38]:
import pandas as pd
df = pd.DataFrame(res)
df

,ajax,feyenoord,psv,fcutrecht,willemII
1,45,21,22,15,7
2,31,38,23,12,6
3,16,20,39,18,17
4,16,23,15,39,17
5,2,8,11,26,63
